#Python Version Update

In [ ]:
!python --version

In [ ]:
# !sudo apt-get update -y
# !sudo apt-get install python3.9

In [ ]:
#change alternatives
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

In [ ]:
# !sudo apt install python3-pip

In [ ]:
# !python --version

# Step-(0): Mount the Drive to colab notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# For removing the default sample data in drive.
!rm -r '/content/sample_data'

# Step-(1) : Import the Image Dataset from the Drive.

In [ ]:
# To Unzip the dataset zip file
!unzip /content/drive/MyDrive/ML-dir/Datasets/rpgo_v1_datasets.zip -d '/content'

# To remove the dir
# !rm -r '/content/rpgo_imgsets_colab'

# Step-(2): Install the Pre-requisites.

In [ ]:
!pip uninstall Cython -y # Temporary fix for "No module named 'object_detection'" error

In [ ]:
# Clone the tensorflow models repository from GitHub
!git clone --depth 1 https://github.com/tensorflow/models
# !rm -r '/content/models'

In [ ]:
# Copy setup files into models/research folder
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
#cp object_detection/packages/tf2/setup.py .

In [ ]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
# Install the Object Detection API
# Need to do a temporary fix with PyYAML because Colab isn't able to install PyYAML v5.4.1
!pip install pyyaml==5.3
!pip install /content/models/research/

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0

In [ ]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

# Step-(3): Set the Path for TF-Record dataset.

In [ ]:
train_record_fname = '/content/rpgo_v1_datasets/rpgo_slant_straight_320_tfrecord/train/train.tfrecord'
val_record_fname = '/content/rpgo_v1_datasets/rpgo_slant_straight_320_tfrecord/valid/valid.tfrecord'
label_map_pbtxt_fname = '/content/rpgo_v1_datasets/rpgo_slant_straight_320_tfrecord/train/label_map.pbtxt'

# Step-(4): Set the Model Configurations.

In [ ]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',

    },
    # The centernet model isn't working as of 9/10/22
    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

# Step-(5): Set the Training steps for model.

In [ ]:
# Set training parameters for the model

# Set the training steps.
num_steps = 1000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 16

In [ ]:
# Set file locations and get number of classes for config file
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)

In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /content/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


In [ ]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = '/content/models/mymodel/pipeline_file.config'
model_dir = '/content/training/'

# Step-(6): Model Training

In [ ]:
# Run training!
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

# Step-(7): Convert Model to TensorFlow SavedModel.

In [ ]:
# Make a directory to store the trained TFLite model
!mkdir /content/custom_model_lite
output_directory = '/content/custom_model_lite'

# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = '/content/training'

!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}


In [ ]:
# !rm -r '/content/custom_model_lite'
# !rm -r '/content/models'
# !rm -r '/content/training'

# Step-(8): Convert SavedModel to Quantized TF-lite Models:

In [ ]:
# Import packages
import tensorflow as tf
import random
import glob

Ordinarily, creating a TensorFlow Lite model is just a few lines of code with [`TFLiteConverter`](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter). For example, this creates a basic (un-quantized) TensorFlow Lite model:

In [ ]:
# To convert a basic (un-quantized) TensorFlow Lite model:

# converter = tf.lite.TFLiteConverter.from_saved_model('/content/custom_model_lite/saved_model')
# tflite_model = converter.convert()

# with open('/content/custom_model_lite/-320-normal.tflite', 'wb') as f:
  # f.write(tflite_model)

However, this `.tflite` file still uses floating-point values for the parameter data, and we need to fully quantize the model to int8 format.

To fully quantize the model, we need to perform [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) with a representative dataset, which requires a few more arguments for the `TFLiteConverter`, and a function that builds a dataset that's representative of the training dataset.

So let's convert the model again with post-training quantization:

In [ ]:
# Get list of all images in train directory
image_path = '/content/rpgo_v1_datasets/rpgo_slant_straight_320_yolov8/train/images'

jpg_file_list = glob.glob(image_path + '/*.jpg')
JPG_file_list = glob.glob(image_path + '/*.JPG')
png_file_list = glob.glob(image_path + '/*.png')
bmp_file_list = glob.glob(image_path + '/*.bmp')

quant_image_list = jpg_file_list + JPG_file_list + png_file_list + bmp_file_list

In [ ]:
imgsz = 320

def representative_data_gen():
  dataset_list = quant_image_list
  quant_num = 300
  for i in range(quant_num):
    pick_me = random.choice(dataset_list)
    image = tf.io.read_file(pick_me)

    if pick_me.endswith('.jpg') or pick_me.endswith('.JPG'):
      image = tf.io.decode_jpeg(image, channels=3)
    elif pick_me.endswith('.png'):
      image = tf.io.decode_png(image, channels=3)
    elif pick_me.endswith('.bmp'):
      image = tf.io.decode_bmp(image, channels=3)

    image = tf.image.resize(image, [imgsz, imgsz])  # TO DO: Replace 300s with an automatic way of reading network input size
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

In [ ]:
# Initialize converter module
converter = tf.lite.TFLiteConverter.from_saved_model('/content/custom_model_lite/saved_model')

# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
# converter.target_spec.supported_types = [tf.int8]
# These set the input tensors to uint8 and output tensors to float32
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model = converter.convert()

with open('/content/custom_model_lite/rpgo-slant-straight-320-int8.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# To Copy the tflite model to drive
# !scp -r '/content/custom_model_lite/mobilenet-v2-640-normal.tflite' '/content/drive/MyDrive/rpgo_colab_models'

# Step-(9): Metadata Writer for TF-lite Model.

In [ ]:
!pip install tflite-support

In [ ]:
import os

from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

In [ ]:
# Creating new metadata dir and copying the model from custom_model_lite dir.
!mkdir /content/Metadata_Written_Tf-lite_Models

In [ ]:
# Copying model to metadata folder
!scp -r '/content/custom_model_lite/rpgo-slant-straight-320-int8.tflite' '/content/Metadata_Written_Tf-lite_Models/rpgo-slant-straight-320-int8.tflite-md.tflite'

In [ ]:
model_path = "/content/Metadata_Written_Tf-lite_Models/rpgo-slant-straight-320-int8.tflite-md.tflite"
label_path = "/content/rpgo_v1_datasets/label_rpgo.txt"

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "MobileNetV2-FPN-lite Object Detection"
model_meta.description = ("Identify the most prominent object in the "
                          "image from a set of 1 categories such as "
                          "rpgo.")
model_meta.version = "v2"
model_meta.author = "TensorFlow"
model_meta.license = "Apache Version 2.0"

# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()

input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(320, 320))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creates output info.
output_location_meta = _metadata_fb.TensorMetadataT()
output_location_meta.name = "location"
output_location_meta.description = "The bbox or locations of the detected boxes."
output_location_meta.content = _metadata_fb.ContentT()
output_location_meta.content.contentPropertiesType = (_metadata_fb.ContentProperties.BoundingBoxProperties) # https://www.tensorflow.org/lite/api_docs/python/tflite_support/metadata_schema_py_generated/ContentProperties
output_location_meta.content.contentProperties = (_metadata_fb.BoundingBoxPropertiesT())
output_location_meta.content.contentProperties.index = [1, 0, 3, 2] # https://www.tensorflow.org/lite/api_docs/python/tflite_support/metadata_schema_py_generated/BoundingBoxType
output_location_meta.content.contentProperties.type = (_metadata_fb.BoundingBoxType.BOUNDARIES)
output_location_meta.content.contentProperties.coordinateType = (_metadata_fb.CoordinateType.RATIO)
output_location_meta.content.range = _metadata_fb.ValueRangeT()
output_location_meta.content.range.min = 2
output_location_meta.content.range.max = 2

output_class_meta = _metadata_fb.TensorMetadataT()
output_class_meta.name = "classes"
output_class_meta.description = "The class or categories of the detected boxes."
output_class_meta.content = _metadata_fb.ContentT()
output_class_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.FeatureProperties)
output_class_meta.content.contentProperties = (
        _metadata_fb.FeaturePropertiesT())
output_class_meta.content.range = _metadata_fb.ValueRangeT()
output_class_meta.content.range.min = 2
output_class_meta.content.range.max = 2
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(label_path)
label_file.description = "Label of objects that this model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS #TENSOR_AXIS_LABELS
output_class_meta.associatedFiles = [label_file]

output_score_meta = _metadata_fb.TensorMetadataT()
output_score_meta.name = "score"
output_score_meta.description = "The scores of the detected boxes."
output_score_meta.content = _metadata_fb.ContentT()
output_score_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.FeatureProperties)
output_score_meta.content.contentProperties = (
        _metadata_fb.FeaturePropertiesT())
output_score_meta.content.range = _metadata_fb.ValueRangeT()
output_score_meta.content.range.min = 2
output_score_meta.content.range.max = 2

output_no_of_detection_meta = _metadata_fb.TensorMetadataT()
output_no_of_detection_meta.name = "number of detections"
output_no_of_detection_meta.description = "The number of the detected boxes."
output_no_of_detection_meta.content = _metadata_fb.ContentT()
output_no_of_detection_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.FeatureProperties)
output_no_of_detection_meta.content.contentProperties = (
        _metadata_fb.FeaturePropertiesT())

# Creates subgraph info.
group = _metadata_fb.TensorGroupT()
group.name = "detection result"
group.tensorNames = [output_location_meta.name, output_class_meta.name, output_score_meta.name]

subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_location_meta, output_class_meta, output_score_meta, output_no_of_detection_meta]
subgraph.outputTensorGroups = [group]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

"""Populates metadata and label file to the model file."""
populator = _metadata.MetadataPopulator.with_model_file(model_path)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([label_path])
populator.populate()

In [ ]:
# !pip install tensorflow

In [ ]:
import tensorflow as tf

tf_mod_path = "/content/Metadata_Written_Tf-lite_Models/rpgo-slant-straight-320-int8.tflite-md.tflite"
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(model_path= tf_mod_path)

# Resize input shape for dynamic shape model and allocate tensor
interpreter.resize_tensor_input(interpreter.get_output_details()[0]['index'], [3, 10])
interpreter.allocate_tensors()

# Get input and output tensors.
# https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter#get_input_details
# https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter#get_output_details

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_data = interpreter.get_tensor(input_details[0]['index'])
output_data = interpreter.get_tensor(output_details[3]['index'])

input_dtype = interpreter.get_input_details()[0]['dtype']
output_dtype = interpreter.get_output_details()[0]['dtype']

input_dtype = interpreter.get_input_details()[0]['dtype']
output_dtype = interpreter.get_output_details()[0]['dtype']

input_shape = interpreter.get_input_details()[0]['shape']
output_shape_0 = interpreter.get_output_details()[0]['shape']
output_shape_1 = interpreter.get_output_details()[1]['shape']
output_shape_2 = interpreter.get_output_details()[2]['shape']
output_shape_3 = interpreter.get_output_details()[3]['shape']



# print(input_details)
# print(output_details)

print("Input Data : ", input_dtype, input_shape)
print("Output Data : ", output_dtype, output_shape_0, output_shape_1, output_shape_2, output_shape_3)

# Step-(10): Test the TF-lite Model.

In [ ]:
# output index
# ------------TF1 index --------------------|--------TF2 index-------|
# StatefulPartitionedCall:3 = [1, 10, 4] # boxes         # 1
# StatefulPartitionedCall:2 = [1, 10]    # classes       # 3
# StatefulPartitionedCall:1 = [1, 10]    # scores        # 0
# StatefulPartitionedCall:0 = [1]        # count         # 2


# Script to run custom TFLite model on test images to detect objects
# Source: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/content/results', txt_only=False):

  # Grab filenames of all images in test folder
  images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Randomly select test images
  images_to_test = random.sample(images, num_test_images)

  # Loop over every image and perform detection
  for image_path in images_to_test:

      # Load image and resize to expected shape [1xHxWx3]
      image = cv2.imread(image_path)
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      imH, imW, _ = image.shape
      image_resized = cv2.resize(image_rgb, (width, height))
      input_data = np.expand_dims(image_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects
      detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))

              cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

              # Draw label
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])


      # All the results have been drawn on the image, now display the image
      if txt_only == False: # "text_only" controls whether we want to display the image results or just save them in .txt files
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12,16))
        plt.imshow(image)
        plt.show()

      # Save detection results in .txt files (for calculating mAP)
      elif txt_only == True:

        # Get filenames and paths
        image_fn = os.path.basename(image_path)
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn +'.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        # Write results to text file
        # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
        with open(txt_savepath,'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return

In [ ]:
# Set up variables for running user's model

PATH_TO_MODEL='/content/Metadata_Written_Tf-lite_Models/rpgo-slant-straight-320-int8.tflite-md.tflite' #/content/tflite_cus/detect.tflite'   # Path to .tflite model file
PATH_TO_IMAGES='/content/rpgo_v1_datasets/rpgo_slant_straight_320_yolov8/valid/images'   # Path to test images folder
PATH_TO_LABELS='/content/rpgo_v1_datasets/label_rpgo.txt'   # Path to labelmap.txt file
min_conf_threshold=0.5   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)
images_to_test = 10   # Number of images to run detection on

# Run inferencing function!
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)

# Step-(11): Calculate mAP

In [ ]:
%%bash
git clone https://github.com/Cartucho/mAP /content/mAP
cd /content/mAP
rm input/detection-results/*
rm input/ground-truth/*
rm input/images-optional/*
wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py

In [ ]:
# Add the pascal voc dataset to this code
!cp /content/rpgo_v2_datasets/rpgo_v2_320_pvoc/valid/* /content/mAP/input/images-optional # Copy images and xml files
!mv /content/mAP/input/images-optional/*.xml /content/mAP/input/ground-truth/  # Move xml files to the appropriate folder

In [ ]:
# Conversion from Pascal VOC xml to txt.
!python /content/mAP/scripts/extra/convert_gt_xml.py

In [ ]:
# Need to remove existing detection results first
# !rm /content/mAP/input/detection-results/*

# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/mAP/input/images-optional'   # Path to test images folder
PATH_TO_MODEL='/content/rpgo-v2-320-int8-md.tflite'   # Path to quantized .tflite model file
PATH_TO_LABELS='/content/rpgo_v2_datasets/label_rpgo.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/mAP/input/detection-results' # Folder to save detection results in

min_conf_threshold=0.1   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(500, len(image_list)) # If there are more than 500 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

In [ ]:
%cd '/content/mAP'

In [ ]:
!python calculate_map_cartucho.py --labels=/content/rpgo_v2_datasets/label_rpgo.txt

In [ ]:
# To Remove the existing files:
!rm /content/mAP/input/detection-results/*
!rm -r /content/mAP/outputs

#Save the Trained Model Checkpoints to Drive.

In [ ]:
# To Remove the Dir from /content
# !rm -r '/content/custom_model_lite'
# !rm -r '/content/models'
# !rm -r '/content/training'
# !rm -r '/content/mobilnetssd-v2-fpn-lite_320_stp_2000'
# !rm -r '/content/rpgo-straight_320_stp_1000'

In [ ]:
!mkdir '/content/rpgo-slant-straight-320_stp_1000'
!mkdir '/content/rpgo-slant-straight-320_stp_1000/custom_model_lite'
!mkdir '/content/rpgo-slant-straight-320_stp_1000/models'
!mkdir '/content/rpgo-slant-straight-320_stp_1000/training'
!mkdir '/content/rpgo-slant-straight-320_stp_1000/Metadata_Written_Tf-lite_Models'

In [ ]:
!scp -r '/content/models' '/content/rpgo-slant-straight-320_stp_1000'

In [ ]:
!scp -r '/content/training' '/content/rpgo-slant-straight-320_stp_1000'

In [ ]:
!scp -r '/content/custom_model_lite' '/content/rpgo-slant-straight-320_stp_1000'

In [ ]:
!scp -r '/content/Metadata_Written_Tf-lite_Models' '/content/rpgo-slant-straight-320_stp_1000'

In [ ]:
!zip -r '/content/rpgo-slant-straight-320_stp_1000.zip' 'rpgo-slant-straight-320_stp_1000'

In [ ]:
# !mkdir '/content/drive/MyDrive/Pre_trained_Models'

In [ ]:
!scp -r '/content/rpgo-slant-straight-320_stp_1000.zip' '/content/drive/MyDrive/ML-dir/Pre_trained_Models'

# Save it Final Models Folder

In [ ]:
!scp -r '/content/Metadata_Written_Tf-lite_Models/rpgo-slant-straight-320-int8-md.tflite' '/content/drive/MyDrive/ML-dir/Final_Models'

# Code Functions:
1. **To make new dir**: !mkdir 'path_to_new_dir'
2. **Secured Copy and paste**: !scp -r 'path_to_copy_from_dir' 'path_to_copy_to_dir'
3. **To Unzip a zip dir**: !unzip 'path_to_zipfolder.zip' -d 'path_to_destination'
4. **To Zip the dir**: !zip -r 'path_to_zip_dir_destination' 'path_to_dir_to_be_zipped'